In [37]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import mplfinance as mpl
import os


#install symbols nasdaq100の銘柄コード

sign =['ATVI', 'AMD', 'ALXN', 'ALGN', 'GOOGL', 'GOOG', 'AMZN', 'AEP', 'AMGN', 'ADI'
        , 'ANSYS', 'ANSS', 'AAPL', 'AMAT', 'ASML', 'TEAM', 'ADSK', 'ADP', 'BIDU', 'BIIB', 'BKNG'
        , 'AVGO', 'CDNS', 'CDW', 'CDW', 'CERN', 'CHTR', 'CHKP', 'CTAS', 'CSCO', 'CTSH', 'CMCSA'
        , 'CPRT', 'COST', 'CSX', 'DXCM', 'DOCU', 'DLTR', 'EBAY', 'EA', 'EXC', 'FB', 'FAST'
        , 'FISV', 'FOXA', 'FOX', 'GILD', 'IDXX', 'ILMN', 'INCY', 'INTC', 'INTU', 'ISRG', 'JD'
        , 'KDP', 'KLAC', 'KHC', 'LRCX', 'LULU', 'MAR', 'MRVL', 'MTCH', 'MXIM', 'MELI', 'MCHP', 'MU'
        , 'MSFT', 'MRNA', 'MDLZ', 'MNST', 'NTES', 'NFLX', 'NVDA', 'NXPI', 'ORLY', 'OKTA', 'PCAR', 'PAYX'
        , 'PYPL', 'PTON', 'PEP', 'PDD', 'QUALCOMM', 'QCOM', 'REGN', 'ROST', 'SGEN', 'SIRI', 'SWKS', 'SPLK'
        , 'SBUX', 'SNPS', 'TMUS', 'TSLA', 'TXN', 'TCOM', 'VRSN', 'VRSK', 'VRTX', 'WBA', 'WDAY', 'XEL', 'XLNX'
        , 'ZM']
totalresult = []

for i in sign:
    stock_code = i 
    start = '2015-01-01'
    end = '2021-02-02'
    df = yf.download(stock_code, start, end, interval='1d') 
    
        
    
    print(i)
    term=80
    # chek the highest price in the past {term} times
    df['Highest'+str(term)] = df.iloc[:, 4].rolling(window=term).max()

    """
    メインループ
    基本戦略
    過去longTerm間の調整済み終値の最高値をその日の調整済み終値が更新し,
    次の日に前日の安値を下回った時点で空売り
    
    その日の終値で損失があれば損切り
 　　次の日の始値で決済
    """
    # 買いポジションは０にする
    sell_position = False
   
    # 損切り率
    stopLoss = 1.04
    
    #買い日と売り日
    buy_days = []
    profit_buy_days = []
    sell_days = []
    second_sell_days = []

    
    percentChange = []
    for i in range(1, len(df)-1):
    
        Highest = df['Highest'+str(term)][i-1]
        high_price = df['High'][i]
        low_price = df['Low'][i]
        tomorrow_low = df['Low'][i+1]
        close = df['Adj Close'][i]   
        open_price = df['Open'][i]
        tomorrow_open = df['Open'][i+1]
        last_close = df['Close'][i-1]
        last_high = df['High'][i-1]
        sellday = pd.to_datetime(df.index[i])

        # avoid NaN data 買いトレンド
        if np.isnan(Highest) == 0:
            if close > Highest and sell_position == 0:
                sell_position = 1
                signal_price = low_price
                signal_day = i
                print('signal')
                
                
        if sell_position == 1 and open_price < signal_price and i == signal_day +1:
            sell_position = 2
            sell_price = open_price
            sellday = pd.to_datetime(df.index[i])
            sell_days.append(sellday)
            print('buy'+str(df.index[i]))
            sell_point = i

        elif sell_position == 1  and low_price < signal_price  and i == signal_day +1:
            sell_position = 2
            sell_price = signal_price
            sellday = pd.to_datetime(df.index[i])
            sell_days.append(sellday)
            sell_point = i
            print('buy'+str(df.index[i]))

            

         # 損切り
        if sell_position == 2 and close > sell_price * 1.03:
            sell_position = 0
            percent = -(close/sell_price - 1) * 100
            percentChange.append(percent)
            buyday = pd.to_datetime(df.index[i])
            buy_days.append(buyday)
            print('損切り'+str(df.index[i]))
            
        # buy at tomorrow opening 
        elif sell_position == 2 and i == sell_point + 1 :
            
            sell_position = 0 
            percent = -(open_price/sell_price - 1) * 100
            percentChange.append(percent)
            buyday = pd.to_datetime(df.index[i])
            buy_days.append(df.index[i])
            print('決済'+str(df.index[i]))
            

        counter += 1


    # statistic
#     ------------------------------------------
    gains = 0
    losses = 0
    numlosses = 0
    numgains = 0
    total_return = 1


    for i in percentChange:
        if i > 0:
            numgains += 1
            gains += i
        else:
            numlosses += 1
            losses += i
        total_return = total_return * ((i / 100) + 1)
        
        
    total_return = round((total_return - 1)*100, 2)
    if numgains > 0:
        ave_gain = gains / numgains
#         max_return = max(percentChange)
    else:
        ave_gain = 0
        max_return = 'unknown'
    
    if numlosses > 0:
        ave_loss = losses / numlosses
#         max_loss = min(percentChange)
        risk_reward_retio = - ave_gain / ave_loss
    else:
        ave_loss = 0
        max_loss = 'unknown'
        risk_reward_retio = 'inf'
    
    if numgains > 0 or numlosses > 0:
        batting_retio = numgains / (numgains + numlosses)
    else:
        batting_retio = 0
    
    trades = numgains + numlosses
    ave_total = ave_gain * batting_retio + ave_loss * (1 - batting_retio)

    results = [numgains+numlosses, round(total_return,2), round(ave_gain,2), round(ave_loss,2)
               ,batting_retio ,ave_total ]
    
    if not results is None:
         totalresult.append(results)
            
#    #買いポイント描写
#     buy_signals = []
#     sell_signals = []
#     for i in df.index:
#         if i in buy_days:
#             buy_signals.append(df['Adj Close'][i])
#         else:
#             buy_signals.append(np.nan)
#         if i in sell_days:
#             sell_signals.append(df['Adj Close'][i])
#         else:
#             sell_signals.append(np.nan)
#     adp = [
#         mpl.make_addplot(buy_signals, type='scatter', markersize=10000, marker='|', color="b"),
#         mpl.make_addplot(sell_signals, type='scatter', markersize=10000, marker='|', color="r")]
#     fig = mpl.plot(df, type='candle',figsize=(30, 15),mav=(short_sma, long_sma, verylong_sma),
#                addplot=adp, style='yahoo', volume=True,  title=str(stock_code))





print(totalresult)

columns = ['trades', 'Total return', 'Average Gain', 'Average Loss', '勝率' ,'Average gain and profit']
df = pd.DataFrame(totalresult, columns=columns, index = sign)
df.to_csv('./result.csv')

print(df)


[*********************100%***********************]  1 of 1 completed
ATVI
signal
[*********************100%***********************]  1 of 1 completed
AMD
signal
[*********************100%***********************]  1 of 1 completed
ALXN
signal
buy2015-06-24 00:00:00
決済2015-06-25 00:00:00
signal
[*********************100%***********************]  1 of 1 completed
ALGN
signal
[*********************100%***********************]  1 of 1 completed
GOOGL
signal
[*********************100%***********************]  1 of 1 completed
GOOG
signal
[*********************100%***********************]  1 of 1 completed
AMZN
signal
[*********************100%***********************]  1 of 1 completed
AEP
signal
[*********************100%***********************]  1 of 1 completed
AMGN
signal
[*********************100%***********************]  1 of 1 completed
ADI
signal
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ANSYS: No data found, symbol may be delisted
ANSY

[*********************100%***********************]  1 of 1 completed
ROST
signal
buy2015-07-23 00:00:00
決済2015-07-24 00:00:00
signal
buy2015-08-06 00:00:00
決済2015-08-07 00:00:00
signal
buy2015-08-12 00:00:00
決済2015-08-13 00:00:00
signal
[*********************100%***********************]  1 of 1 completed
SGEN
signal
[*********************100%***********************]  1 of 1 completed
SIRI
signal
buy2015-08-14 00:00:00
決済2015-08-17 00:00:00
signal
buy2015-10-21 00:00:00
決済2015-10-22 00:00:00
signal
[*********************100%***********************]  1 of 1 completed
SWKS
signal
[*********************100%***********************]  1 of 1 completed
SPLK
signal
[*********************100%***********************]  1 of 1 completed
SBUX
signal
[*********************100%***********************]  1 of 1 completed
SNPS
signal
[*********************100%***********************]  1 of 1 completed
TMUS
signal
buy2015-04-30 00:00:00
決済2015-05-01 00:00:00
signal
buy2015-05-19 00:00:00
決済2015-05-20 00:0